# Check MRI Quality

Things that need to be adjusted for SEA Lab:
* File paths (of course)
* Modify to account for multiple runs of a sequence (need to know what that looks like scanner side)


In [ ]:
# Participant-specific information
subject_id = '999'

mri_location = '/Users/catcamacho/Box/Vandy-Stanford MR comparison/Vanderbilt/'

raw_data_locations = {'rest': mri_location + subject_id + '/RAW/*rsEPI-PE0_*', 
                      'rest': mri_location + subject_id + '/RAW/*rsEPI-PE1_*', 
                      'dwi0': mri_location + subject_id + '/NIFTI/*DWI_MB3R1_pe0*', 
                      'dwi1': mri_location + subject_id + '/NIFTI/*DWI_MB3R1_pe1*',
                      'dwi': mri_location + subject_id + '/NIFTI/*DTI_b700_2000_96dir*',
                      'qt1': mri_location + subject_id + '/NIFTI/*qT1*', 
                      't1w': mri_location + subject_id + '/NIFTI/*AnatBrain_T13D*',
                      't2w': mri_location + subject_id + '/NIFTI/*3D_Brain_VIEW_T2*', 
                      'pcasl_pw': mri_location + subject_id + '/NIFTI/*pCASL_PLD1900_LD1000_REL5FIXED.01_real*', 
                      'pcasl_m0': mri_location + subject_id + '/NIFTI/*pCASL_REL5FIXED_M0.01*'}

In [1]:
subject_id = 'newborn-1019'
path = '/Users/catcamacho/Box/SEA/raw/newborn-1019/'

from nipype.interfaces.dcm2nii import Dcm2niix
convert = Dcm2niix()
convert.bids_format=True
convert.inputs.source_names = [path + 'raw/Humphreys_136164_WIP_rsEPI-PE0_MB2R_testIM_21_1.PAR']
convert.inputs.output_dir= path + 'recon'
convert.run()

181218-10:13:40,463 interface INFO:
	 stderr 2018-12-18T10:13:40.463387:Error: Unknown type 30: not magnitude[0], real[1], imaginary[2] or phase[3].
181218-10:13:40,465 interface INFO:
	 stdout 2018-12-18T10:13:40.465703:Chris Rorden's dcm2niiX version v1.0.20180622 (JP2:OpenJPEG) GCC6.1.0 (64-bit MacOS)
181218-10:13:40,466 interface INFO:
	 stdout 2018-12-18T10:13:40.465703:Found 51 DICOM file(s)
181218-10:13:40,467 interface INFO:
	 stdout 2018-12-18T10:13:40.465703:Warning: dcm2niix PAR is not actively supported (hint: use dicm2nii)
181218-10:13:40,467 interface INFO:
	 stdout 2018-12-18T10:13:40.465703:Done reading PAR header version 4.2, with 21 slices
181218-10:13:40,468 interface INFO:
	 stdout 2018-12-18T10:13:40.465703:Convert 1 DICOM as /Users/catcamacho/Box/SEA/raw/newborn-1019/recon/raw_sASLperfusion_20181217210905_16_ph (80x80x21x1)
181218-10:13:40,479 interface INFO:
	 stdout 2018-12-18T10:13:40.479308:compress: "/Applications/MRIcroGL/pigz_mricron" -n -f -6 "/Users/catca

### Reorganize and rename data to conform to BIDS

In [ ]:
# Import libraries and set up static study variables
import os
import shutil
from glob import glob
from subprocess import check_call
from nipype.interfaces.dcm2nii import Dcm2niix

sequence_list = list(raw_data_locations.keys())

# make level 1 folders
os.mkdir(mri_location + subject_id + '/anat')
os.mkdir(mri_location + subject_id + '/func')
os.mkdir(mri_location + subject_id + '/dwi')
os.mkdir(mri_location + subject_id + '/qc')

for sequence in sequence_list:
    if 'dwi' in sequence:
        dwis0 = sorted(glob(raw_data_locations['dwi0']))
        if len(dwis0)>0:
            for file in dwis0:
                filepath, filebasename = os.path.split(file)
                filename, fileext = os.path.splitext(filebasename)
                os.rename(file, mri_location + subject_id + '/dwi/'+subject_id+'_dwi_pe0_1'+fileext)
            dwis1 = sorted(glob(raw_data_locations['dwi1']))
            for file in dwis1:
                filepath, filebasename = os.path.split(file)
                filename, fileext = os.path.splitext(filebasename)
                os.rename(file, mri_location + subject_id + '/dwi/'+subject_id+'_dwi_pe1_1'+fileext)
            dwis = sorted(glob(raw_data_locations['dwi']))
            for file in dwis:
                filepath, filebasename = os.path.split(file)
                filename, fileext = os.path.splitext(filebasename)
                os.rename(file, mri_location + subject_id + '/dwi/'+subject_id+'_dwi_pe0_b2000_b700_1'+fileext)
    elif 'rest' in sequence:
        rest0s = sorted(glob(raw_data_locations['rest0']))
        rest1s = sorted(glob(raw_data_locations['rest1']))
        rests = rest0s + rest1s
        for file in rests:
            filepath, filebasename = os.path.split(file)
            shutil.move(file, mri_location + subject_id + '/' + filebasename)    
        convert = Dcm2niix()
        convert.bids_format=True
        convert.inputs.source_dir = mri_location + subject_id
        convert.run()
        check_call('mv', mri_location + subject_id + '/*rsEPI*',mri_location + subject_id +'/func')
    elif 'pcasl' in sequence:
        pws = sorted(glob(raw_data_locations['pcasl_pw']))
        if len(pws)>0:
            for file in pws:
                filepath, filebasename = os.path.split(file)
                filename, fileext = os.path.splitext(filebasename)
                os.rename(file, mri_location + subject_id + '/func/'+subject_id+'_pcasl_pw_1'+fileext)
            m0s = sorted(glob(raw_data_locations['pcasl_m0']))
            for file in m0s:
                filepath, filebasename = os.path.split(file)
                filename, fileext = os.path.splitext(filebasename)
                os.rename(file, mri_location + subject_id + '/func/'+subject_id+'_pcasl_m0_1'+fileext)
    elif sequence=='t1w':
        t1ws = sorted(glob(raw_data_locations['t1w']))
        if len(t1ws)>0:
            for file in t1ws:
                filepath, filebasename = os.path.split(file)
                filename, fileext = os.path.splitext(filebasename)
                os.rename(file, mri_location + subject_id + '/anat/'+subject_id+'_T1w_1'+fileext)
    elif sequence=='t2w':
        t2ws = sorted(glob(raw_data_locations['t2w']))
        if len(t2ws)>0:
            for file in t2ws:
                filepath, filebasename = os.path.split(file)
                filename, fileext = os.path.splitext(filebasename)
                os.rename(file, mri_location + subject_id + '/anat/'+subject_id+'_T2w_1'+fileext)
    else sequence=='qt1':
        qt1s = sorted(glob(raw_data_locations['qt1']))
        if len(qt1s)>0:
            for file in qt1s:
                filepath, filebasename = os.path.split(file)
                filename, fileext = os.path.splitext(filebasename)
                os.rename(file, mri_location + subject_id + '/anat/'+subject_id+'_quantitative_T1_1'+fileext)
        
# zip up all the niftis to take up less space
niftis = glob(mri_location + subject_id + '/*/*.nii')
for nii in niftis:
    check_call(['gzip',nii]) 
    
# Tar up the extra sequences to take up less space
os.rename(mri_location + subject_id +'/NIFTI', mri_location + subject_id +'/misc')
os.rename(mri_location + subject_id +'/XMLPARREC', mri_location + subject_id +'/raw')
check_call(['tar','-zcvf',mri_location + subject_id +'/misc.tgz',mri_location + subject_id +'/misc'])
check_call(['tar','-zcvf',mri_location + subject_id +'/raw.tgz',mri_location + subject_id +'/raw'])


## Produce QC Report
This section does the following:
* Create motion summaries for rest and DTI
* create histograms for ASL and qT1 values

In [ ]:
data_locations = {'rest': mri_location + subject_id + '/func/' + subject_id + '_rest_pe0*.nii.gz', 
                  'dwi': mri_location + subject_id + '/dwi/' + subject_id + '_dwi_pe0_b2000_b700*.nii.gz',
                  'qt1': mri_location + subject_id + '/anat/' + subject_id + '_quantitative_T1.nii.gz',
                  't1w': mri_location + subject_id + '/anat/' + subject_id + '_T1w.nii.gz',
                  't2w': mri_location + subject_id + '/anat/' + subject_id + '_T2w.nii.gz', 
                  'pcasl_pw': mri_location + subject_id + '/func/' + subject_id + '_pcasl_pw.nii.gz', 
                  'pcasl_m0': mri_location + subject_id + '/func/' + subject_id + '_pcasl_m0.nii.gz'}

# Sort what type of QC needs to be conducted on each
motion_imgs = ['rest','dwi']

In [ ]:
from nipype.interfaces.fsl import MotionOutliers
from numpy import loadtxt

TR=0.8

# Get frame-wise displacement for each run
get_FD = MotionOutliers()
get_FD.inputs.metric = 'fd'
get_FD.inputs.no_motion_correction=False
get_FD.inputs.threshold=0.25

for img in motion_imgs:
    get_FD.inputs.in_file = data_locations[img]
    get_FD.inputs.out_metric_values = mri_location + subject_id + '/qc/'+img+'_FD.txt'
    get_FD.inputs.out_metric_plot = mri_location + subject_id + '/qc/'+img+'_motionplot.png'
    get_FD.run()
    
    motion = loadtxt(mri_location + subject_id + '/qc/'+img+'_FD.txt')
    not_usable = motion[motion>0.25]
    usable_vols = len(motion)-len(not_usable)
    results_file = open(mri_location + subject_id + '/qc/'+img+'_usability_summary.txt','w')
    results_file.write('Out of %f collected volumes, %f exceeded 0.25mm framewise displacement.' % (len(motion),len(not_usable)))
    if img=='rest':
        results_file.write('Usable volumes: %f; Usable minutes: %f.' % (usable_vols, (usable_vols*TR)/60))
    results_file.close()    
    

In [ ]:
## Create histogram of qT1 values
from nibabel import load
from matplotlib import pyplot as plt

qT1_image = load(data_locations['qt1'])
qT1_data = qT1_image.get_data()

plt.Figure()
data = qT1_data.flatten()
plt.hist(data, bins = range(100,4600,100))
plt.title("Distribution of T1 values for subject " + subject)
plt.savefig(filepath + '/' + subject_id + '_T1_histogram.png')
plt.show()